In [ ]:
from music21 import converter, environment, tempo, metadata
import os
import glob
from PIL import Image
import platform
from pathlib import Path


us = environment.UserSettings()
# ======================================================================================
# Added multiple paths to accomodate different systems
if platform.system() == 'Windows':
    # Paths for MS3 and 4
    candidates = [
        r"C:\Program Files\MuseScore 4\bin\MuseScore4.exe",
        r"C:\Program Files\MuseScore 3\bin\MuseScore3.exe"
    ]
elif platform.system() == 'Darwin':  # macOS
    candidates = [
        "/Applications/MuseScore 4.app/Contents/MacOS/mscore",
        "/Applications/MuseScore 3.app/Contents/MacOS/mscore"
    ]
else:  # Linux
    candidates = [
        "/usr/bin/musescore4",
        "/usr/bin/musescore3",
        "/usr/bin/mscore"
    ]

for path in candidates:
    if os.path.exists(path):
        us['musicxmlPath'] = path
        us['musescoreDirectPNGPath'] = path
        break
else:
    print("No MuseScore path found, please check your installation or used path.")
# Old Code
# us = environment.UserSettings()
# us['musicxmlPath'] = '/usr/bin/musescore3'

# =========================================
# Summary of needed directorys
project_root = Path(os.getcwd()).resolve().parent
output_dir = project_root / "output" / "model_midi"

print("📂 MIDI-Folder:", output_dir)
print("✅ exists:", output_dir.exists())
print("📄 data:", list(output_dir.glob("*.mid")))

print(environment.UserSettings())

📂 MIDI-Folder: C:\Users\Basti\Desktop\Machine_Learning\Project\Keio_Music_Transcription\output\model_midi
✅ exists: True
📄 data: [WindowsPath('C:/Users/Basti/Desktop/Machine_Learning/Project/Keio_Music_Transcription/output/model_midi/OMORI_Final-Duet_basic_pitch.mid')]
🎼 Found Midi Data: ['C:\\Users\\Basti\\Desktop\\Machine_Learning\\Project\\Keio_Music_Transcription/output/model_midi\\OMORI_Final-Duet_basic_pitch.mid']
📁 directory path:
 c:\Users\Basti\Desktop\Machine_Learning\Project\Keio_Music_Transcription\notebooks\output\model_midi
{'directoryScratch': None, 'lilypondPath': 'c:\\Users\\Basti\\Desktop\\Machine_Learning\\Project\\Keio_Music_Transcription\\notebooks\\lilypond', 'lilypondVersion': None, 'lilypondFormat': 'pdf', 'lilypondBackend': 'ps', 'musicxmlPath': 'C:\\Program Files\\MuseScore 3\\bin\\MuseScore3.exe', 'midiPath': None, 'graphicsPath': None, 'vectorPath': None, 'pdfPath': None, 'braillePath': None, 'musescoreDirectPNGPath': 'C:\\Program Files\\MuseScore 3\\bin\\Mu

In [24]:
def add_margins_and_white_bg(image_path, top=20, right=60, bottom=20, left=60):
    img = Image.open(image_path).convert("RGBA")
    new_width = img.width + left + right
    new_height = img.height + top + bottom
    background = Image.new("RGBA", (new_width, new_height), (255, 255, 255, 255))
    background.paste(img, (left, top), mask=img)
    background.convert("RGB").save(image_path, "PNG")

def fix_all_images(png_prefix):
    png_files = sorted(glob.glob(f"{png_prefix}-*.png"))
    if os.path.exists(f"{png_prefix}.png"):
        png_files.insert(0, f"{png_prefix}.png")
    
    for path in png_files:
        add_margins_and_white_bg(path)



In [ ]:
midi_path = '../data/Boccherini-Minuet-Flute-and-Piano.mid'

# ===================================================================================
# Changes made: nothing really just wanted to test your programm for a Model Output, 
# but for some reason it doesn't really work. Still your file works idk 
song_name = 'OMORI_Final-Duet'
output_dir = Path.cwd().parent / "output" # for some reason the output_dir defined in the above section didnt work
output_dir.mkdir(exist_ok=True)             
#midi_path = output_dir / f"model_midi" / f"{song_name}_basic_pitch.mid"

print(midi_path)
if not Path(midi_path).exists():
     raise FileNotFoundError(f"🚫 Datei nicht gefunden: {midi_path}")
print(f"Midi-Data Path:{midi_path}")
# ===================================================== End added Code 
score = converter.parse(midi_path)

tempi = list(score.recurse().getElementsByClass(tempo.MetronomeMark))
if len(tempi) > 1:
    for tm in tempi[1:]:
        parent = tm.getContextByClass('Measure')
        if parent:
            parent.remove(tm)
        else:
            tm.activeSite.remove(tm)

filename = os.path.splitext(os.path.basename(midi_path))[0]
if score.metadata is None:
    score.metadata = metadata.Metadata()
score.metadata.title = filename

output_path = f"../output/{filename}"
score.write('musicxml.png', fp=f"{output_path}.png")

fix_all_images(output_path)



../data/Boccherini-Minuet-Flute-and-Piano.mid
Midi-Data Path:../data/Boccherini-Minuet-Flute-and-Piano.mid
